In [1]:
%load_ext autoreload
%autoreload 2

import os
from utils import set_seed
import torch

set_seed(1234)
os.environ['HF_HOME'] = 'hf_cache' # Don't want model files in our home directory due to disk quota

torch.cuda.is_available()

True

In [2]:
from transformers import LukeTokenizer
import const
from model import DocRedModel
import json

MODEL_NAME = const.LUKE_LARGE

rel2id = json.load(open('data/meta/rel2id.json'))
id2rel = {v: k for k, v in rel2id.items()}
tokenizer = LukeTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True) # Prefix space since we are doing word-level tokenization in read_docred (docred is weird)

/data2/nhanse02/thesis/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
from utils import read_docred
import json

train_samples = read_docred(fp='data/train_annotated.json', rel2id=rel2id, tokenizer=tokenizer)
dev_samples = read_docred(fp='data/dev.json', rel2id=rel2id, tokenizer=tokenizer)

/data2/nhanse02/thesis/data/dev.json: 100%|██████████| 998/998 [00:07<00:00, 136.02it/s]


In [5]:
from utils import get_holdouts, remove_holdouts

holdout_rel_batches, holdout_id_batches = get_holdouts(train_samples=train_samples,
                                                       dev_samples=dev_samples,
                                                       rel2id=rel2id,
                                                       id2rel=id2rel)


holdout_rels, holdout_ids = holdout_rel_batches[0], holdout_id_batches[0] 
train_samples, rel2id_holdout, id2rel_holdout = remove_holdouts(samples=train_samples,
                                                                holdout_rels=holdout_rels, # NOTE: only testing with first batch
                                                                rel2id=rel2id,
                                                                id2rel=id2rel)